In [2]:
import sys
# sys.path.append("D:\thesis\thesis\rllib")
import os
import rllib as rl
import json
import joblib
import numpy as np
import pandas as pd
dataset_train = "./ML/output/random2/symptoms/csv/symptoms.csv"
dataset_test = "./data_preprossing/output/test_cleaned.csv"
symptom_map_file = "./data/basic/symptoms_db.json"
condition_map_file = "./data/basic/conditions_db.json"
group_file = "./data/basic/conditions_group.json"
test = "./data_preprossing/output/data/total_1k.csv"

In [4]:
output_path = "./data_preprossing/rl/"

classlabel = [24, 29, 49, 38, 5, 3, 43, 48, 41, 37, 12, 31, 9, 42, 35, 28, 8, 14, 36, 4, 40, 19, 20, 21, 39, 53, 10, 25, 0, 44, 2, 17, 45]
group_id = [1,2,3,4,5,6,7,8,9]
symptom_columns = ['Index','GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
df = pd.read_csv(dataset_train,sep = "\t")
with open(symptom_map_file) as fp:
        symptoms_db = json.load(fp)

with open(condition_map_file) as fp:
        conditions_db = json.load(fp)

with open(group_file) as  fp:
    group_db = json.load(fp)
print(group_db)
# groups = dict(map(reversed,group_db.items()))
# print(groups)

# def transform_symptoms_nlice_adv(symptom_str):  
#     # print(symptom_str)
#     symptom_list = symptom_str.split(";")
#     transformed_symptoms = []
#     for _symp_def in symptom_list:
#         # print(_symp_def)
#         sym_list = _symp_def.split(":")
#         if(len(sym_list)==9):
#             _symptom, _nature, _location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")
#             to_transform = [
#                 ":".join([str(_symptom)]),
#                 ":".join([str(_nature)]),
#                 ":".join([str(_location)]),
#                 ":".join([str(_intensity)]),
#                 ":".join([str(_duration)]),
#                 ":".join([str(_onset)]),
#                 ":".join([str(_exciation)]),
#                 ":".join([str(_frequency)])
#                 ]
#             transformed_str = ";".join(to_transform)
#             transformed_symptoms.append(transformed_str)

#     return ";".join(transformed_symptoms)
for i in group_id:
    # print(conditions_db)
    conditions = dict(map(reversed, conditions_db.items()))
    groups = dict(map(reversed,group_db.items()))
    # print(groups)
    symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
    df = pd.read_csv(dataset_train,names=symptom_columns)
    label = conditions.get(i)
    df = df.loc[df['PATHOLOGY'] == i]
    df.index.name = "Index"
    df = df[symptom_columns]
#     df = df.loc[df['NUM_SYMPTOMS']>=2]
    df['NUM_SYMPTOMS'] = df.NUM_SYMPTOMS.apply(lambda n: int(n) if int(n) >=2 else np.NAN)
    df['NUM_SYMPTOMS'].replace('', np.nan, inplace=True)
    df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=2 else np.NAN)
    df['SYMPTOMS'].replace('', np.nan, inplace=True)
    df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
    df.dropna(subset=['SYMPTOMS'],inplace=True)
    df = df.drop_duplicates()
    if(len(df)>=2):
        output_file = os.path.join(output_path, "group"+label,"%s.csv" % label)
        df.to_csv(output_file)

{'Acute pancreatitis': 2, 'Acute pyelonefritis': 7, 'Appendicitis': 2, 'Asthma': 1, 'Atrophic gastritis': 2, 'Bacterial meningitis': 4, 'Bladder cancer': 9, 'Breast cancer  female': 9, 'COPD': 1, 'Carpal tunnel syndrome': 5, 'Cauda equina': 5, 'Colon cancer': 9, 'Covid-19': 1, 'Female cystitis': 7, 'Functional constipation': 2, 'Genital herpes  female': 8, 'Genital herpes  male': 8, 'Giardiasis': 2, 'Gonorrhea, feminine': 8, 'Gout': 3, 'Hernia Nuclei Pulposi': 5, 'IBS (Constipation)': 2, 'Iron deficiency anaemia': 10, 'Lower urinary tract infection': 7, 'Lumbago': 5, 'Lumbar spinal stenosis': 5, 'Lung cancer': 9, 'Mastoiditis': 6, 'Migraine': 4, 'Norovirus gastroenteritis': 2, 'Oesophageal carcinoma': 9, 'Osteoarthritis': 3, 'Otitis externa': 6, 'Otitis media acuta': 6, 'Ovarian cancer': 9, 'Pneumococcal pneumonia': 1, 'Pneumothorax': 1, 'Pulmonary edema  old': 1, 'Pulmonary edema  oldest': 1, 'Pulmonary edema  young': 1, 'Rheumatoid arthritis  feminine': 3, 'Rheumatoid arthritis  masc

D:\programe\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [14]:
output_path = "./data_preprossing/rl/"
def extract_symptom_bygroup(filepath):
    use_cols =['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'] 
    df = pd.read_csv(filepath)
    # with open(symptom_db) as fp:
    #     symptoms_mp = json.load(fp)
    symptoms = df['SYMPTOMS']
    condition = df['PATHOLOGY']
    conditions_db = {}
    count1= 0
    for con in condition:
        if con not in conditions_db:
            conditions_db[con] = count1
            count1+=1

    symptoms_db = {}
    count = 0
    for sym in symptoms:
        # print(sym)
        symptom_list = sym.split(";")
        for _symp_def in symptom_list:
            sym_list = _symp_def.split(":")
            _symptom, _nature, _location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")
            if(_symptom == "Alterred_stool"):
                _symptom ="Altered_stool"
            if(_symptom == "Nausea_"):
                _symptom="Nausea"
            if(_symptom == "Pain_relief_"):
                _symptom= "Pain_relief"
            if(_symptom == "Vomitting"):
                _symptom== "Vomiting"
            if(_symptom == "Incontinence_"):
                _symptom="Incontinence"
            
            if _symptom not in symptoms_db:
                symptoms_db[_symptom] = count
                count+=1
    return symptoms_db,conditions_db

for id in [1,2,3,4,5]:
    file = output_path+"group"+str(id)+"/"+str(id)+".csv"
    all_symtom,all_condition = extract_symptom_bygroup(file)
    all_symtom_file = output_path+"group"+str(id)+"/"+"symptom"+str(id)+".json"
    with open(all_symtom_file, "w") as fp:
        sorted_symptoms = sorted(all_symtom.keys())
        symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
        json.dump(all_symtom, fp, indent=4)

    conditon_file = output_path+"group"+str(id)+"/"+"condition"+str(id)+".json"
    with open(conditon_file, "w") as fp:
        sorted_condition = sorted(all_condition.keys())
        condition_map = {code: idx for idx, code in enumerate(sorted_condition)}
        json.dump(all_condition, fp, indent=4) 

In [10]:

group_map = {}
print(group_db)
for k,v in group_db.items():
    if v in group_map.keys():
        group_map[v].append(k)
    else:
      group_map[v] = []
      group_map[v].append(k)
print(group_map)

output_path = "./data_preprossing/rl/"
for id, vs in group_map.items():
    df = pd.read_csv(test,names=symptom_columns)
    print(vs)
    df = df.loc[df['PATHOLOGY'].isin(vs)]
    symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
    df.index.name = "Index"
    df = df[symptom_columns]
#     df = df.loc[df['NUM_SYMPTOMS']>=2]
    # df['NUM_SYMPTOMS'] = df.NUM_SYMPTOMS.apply(lambda n: int(n) if int(n) >=2 else np.NAN)
    df['NUM_SYMPTOMS'].replace('', np.nan, inplace=True)
    df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=2 else np.NAN)
    df['SYMPTOMS'].replace('', np.nan, inplace=True)
    df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
    df.dropna(subset=['SYMPTOMS'],inplace=True)
    df = df.drop_duplicates()
    if(len(df)>=2):

        filegroup = output_path+"group"+str(id)
        if not os.path.exists(filegroup):
            os.makedirs(filegroup)
        output_file = os.path.join(filegroup, "%s.csv" % id)
        df.to_csv(output_file)



{'Acute pancreatitis': 2, 'Acute pyelonefritis': 7, 'Appendicitis': 2, 'Asthma': 1, 'Atrophic gastritis': 2, 'Bacterial meningitis': 4, 'Bladder cancer': 9, 'Breast cancer  female': 9, 'COPD': 1, 'Carpal tunnel syndrome': 5, 'Cauda equina': 5, 'Colon cancer': 9, 'Covid-19': 1, 'Female cystitis': 7, 'Functional constipation': 2, 'Genital herpes  female': 8, 'Genital herpes  male': 8, 'Giardiasis': 2, 'Gonorrhea, feminine': 8, 'Gout': 3, 'Hernia Nuclei Pulposi': 5, 'IBS (Constipation)': 2, 'Iron deficiency anaemia': 10, 'Lower urinary tract infection': 7, 'Lumbago': 5, 'Lumbar spinal stenosis': 5, 'Lung cancer': 9, 'Mastoiditis': 6, 'Migraine': 4, 'Norovirus gastroenteritis': 2, 'Oesophageal carcinoma': 9, 'Osteoarthritis': 3, 'Otitis externa': 6, 'Otitis media acuta': 6, 'Ovarian cancer': 9, 'Pneumococcal pneumonia': 1, 'Pneumothorax': 1, 'Pulmonary edema  old': 1, 'Pulmonary edema  oldest': 1, 'Pulmonary edema  young': 1, 'Rheumatoid arthritis  feminine': 3, 'Rheumatoid arthritis  masc

In [3]:
output_path = "./data_v2/"
def extract_symptom_bygroup(filepath):
    use_cols =['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'] 
    df = pd.read_csv(filepath)
    # with open(symptom_db) as fp:
    #     symptoms_mp = json.load(fp)
    symptoms = df['SYMPTOMS']
    condition = df['PATHOLOGY']
    conditions_db = {}
    count1= 0
    for con in condition:
        if con not in conditions_db:
            conditions_db[con] = count1
            count1+=1

    symptoms_db = {}
    count = 0
    for sym in symptoms:
        # print(sym)
        symptom_list = sym.split(";")
        for _symp_def in symptom_list:
            sym_list = _symp_def.split(":")
            _symptom, _nature, _location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")
            if(_symptom == "Alterred_stool"):
                _symptom ="Altered_stool"
            if(_symptom == "Nausea_"):
                _symptom="Nausea"
            if(_symptom == "Pain_relief_"):
                _symptom= "Pain_relief"
            if(_symptom == "Vomitting"):
                _symptom== "Vomiting"
            if(_symptom == "Incontinence_"):
                _symptom="Incontinence"
            if _symptom not in symptoms_db:
                symptoms_db[_symptom] = count
                count+=1
    return symptoms_db,conditions_db

# for id in [1,2,3,4,5]:
#     file = output_path+"group"+str(id)+"/"+str(id)+".csv"
#     all_symtom,all_condition = extract_symptom_bygroup(file)
#     all_symtom_file = output_path+"group"+str(id)+"/"+"symptom"+str(id)+".json"
#     with open(all_symtom_file, "w") as fp:
#         sorted_symptoms = sorted(all_symtom.keys())
#         symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
#         json.dump(all_symtom, fp, indent=4)

#     conditon_file = output_path+"group"+str(id)+"/"+"condition"+str(id)+".json"
#     with open(conditon_file, "w") as fp:
#         sorted_condition = sorted(all_condition.keys())
#         condition_map = {code: idx for idx, code in enumerate(sorted_condition)}
#         json.dump(all_condition, fp, indent=4) 
    
all_symtom,all_condition = extract_symptom_bygroup(dataset_train)
conditon_file = output_path+"condition"+".json"
with open(conditon_file, "w") as fp:
        sorted_condition = sorted(all_condition.keys())
        condition_map = {code: idx for idx, code in enumerate(sorted_condition)}
        json.dump(all_condition, fp, indent=4) 
symptom_file = output_path+"symptom"+".json"
with open(symptom_file, "w") as fp:
        sorted_symptoms = sorted(all_symtom.keys())
        symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
        json.dump(all_symtom, fp, indent=4) 


ParserError: Error tokenizing data. C error: Expected 9 fields in line 29525, saw 17


In [11]:
output_path = "./data_preprossing/output/data/"
data1k = "./data_preprossing/output/data/total_1k.csv"
classlabel = [24, 29, 49, 38, 5, 3, 43, 48, 41, 37, 12, 31, 9, 42, 35, 28, 8, 14, 36, 4, 40, 19, 20, 21, 39, 53, 10, 25, 0, 44, 2, 17, 45]
symptom_columns = ['Index','GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
df = pd.read_csv(dataset_train,names=symptom_columns)
# df = pd.read_csv(data1k,names=symptom_columns)
df.index.name = "Index"
#     df = df.loc[df['NUM_SYMPTOMS']>=2]
print(df)
# df['NUM_SYMPTOMS'] = df.NUM_SYMPTOMS.apply(lambda n: int(n) if int(n) >=3 else np.NAN)
# df['NUM_SYMPTOMS'].replace('', np.nan, inplace=True)
df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=5 else np.NAN)
df['SYMPTOMS'].replace('', np.nan, inplace=True)
df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
df.dropna(subset=['SYMPTOMS'],inplace=True)
df = df.drop_duplicates()
label = "total_1k_min5"
output_file = os.path.join(output_path, "%s.csv" % label)
df.to_csv(output_file)

D:\programe\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


        GENDER   RACE  AGE_BEGIN                                 PATHOLOGY  \
Index                                                                        
Index   GENDER   RACE  AGE_BEGIN                                 PATHOLOGY   
0            F  white         51                                   Lumbago   
1            F  white         16                 Norovirus gastroenteritis   
2            M  white         71  Viral meningitis  varicella zoster virus   
3            F  white          4                   Pulmonary edema  oldest   
...        ...    ...        ...                                       ...   
355216       M  white         69                                      Gout   
355217       F  white         49                                      COPD   
355218       M  white         80                                    Asthma   
355219       M  white         88         Yersinia enterocolitica infection   
355220       F  white         46            Rheumatoid arthritis

In [ ]:
import os, glob
import pandas as pd

path = "./data_preprossing/output/datas"
output = "./data_preprossing/output/data"

files = os.listdir(path)  #获取文件夹下所有文件名

df1 = pd.read_csv(path + '/' + files[0],encoding='gbk')  #读取首个csv文件，保存到df1中
df1 = df1.iloc[0:500]
for file in files[1:]:     
  df2 = pd.read_csv(path +'/' +  file,encoding='gbk')  #打开csv文件，注意编码问题，保存到df2中
  df2 = df2.iloc[0:500] # first five rows of dataframe
  df1 = pd.concat([df1,df2],axis=0,ignore_index=True)  #将df2数据与df1合并

df1 = df1.drop_duplicates()   #去重
# df1 = df1.reset_index(drop=True) #重新生成index
df1.to_csv(output + '/' + 'total_500.csv') #将结果保存为新的csv文件

In [ ]:
import pathlib
path = "./data_preprossing/output/datas"
from sklearn.model_selection import StratifiedShuffleSplit
def split_data(symptom_file, output_path, use_headers=False, train_split=0.8):
    use_cols =['Delete','GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'] 

    # pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

    if use_headers:
        df = pd.read_csv(symptom_file, names=use_cols)
    else:
        df = pd.read_csv(symptom_file)
    df.index.name = "Index" 
    labels = df['PATHOLOGY']
    df.drop('Delete',axis = 1,inplace = True)
    df.to_csv(symptom_file)
    # datas = df
    # splitter = StratifiedShuffleSplit(1, train_size=train_split)
    # train_index = None
    # test_index = None
    # for tr_idx, tst_index in splitter.split(datas, labels):
    #     train_index = tr_idx
    #     test_index = tst_index
    #     break

    # train_df = df.iloc[train_index]
    # test_df = df.iloc[test_index]

    # train_op = os.path.join(output_path, "RL_train.csv")
    # test_op = os.path.join(output_path, "RL_test.csv")
    # train_df.to_csv(train_op)
    # test_df.to_csv(test_op)
 
    # return train_op,test_op

In [ ]:
op_data_dir = os.path.join(path, "data")
# split into train and test
split_data("./data_preprossing/output/data/total_100.csv", op_data_dir,True,train_split=0.7)

In [3]:
def extract_symptom_nlice(filepath):
    use_cols =['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'] 
    df = pd.read_csv(filepath)
    # with open(symptom_db) as fp:
    #     symptoms_mp = json.load(fp)
    symptoms = df['SYMPTOMS']
    symptoms_db = {}
#     excitation_encoding={
#         "other":1
#     }
#     vas_encoding = {"other": 1}
#     nature_encoding = {}
#     frequency_encoding = {
#     "other": 1}
#     duration_encoding = {
#     "other": 1}
#     onset_encoding = {
#     "other": 1
# }
#     location_encoding={
#         "other":1
#     }
    nature=[]
    location=[]
    location_main=set([])
    vas= []
    excitation= []
    frequency= []
    onset=[]
    duration=[]

    count = 0
    count_nlice = 2
    for sym in symptoms:
        # print(sym)
        symptom_list = sym.split(";")
        for _symp_def in symptom_list:
            sym_list = _symp_def.split(":")
            _symptom, _nature, _location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")
            if _symptom not in symptoms_db:
                symptoms_db[_symptom] = count
                count+=1
            if _nature not in nature:
                nature.append(_nature)          
            if _location not in location:
                location.append(_location)
            if _frequency not in frequency:
                frequency.append(_frequency)
            if _exciation not in excitation:
                excitation.append(_exciation)
            if _intensity not in vas:
                vas.append(_intensity)
            if _onset not in onset:
                onset.append(_onset)
            if _duration not in duration:
                duration.append(_duration)
    return symptoms_db,nature,location,frequency,excitation,vas,onset,duration
excitation_encoding={}
vas_encoding = {}
nature_encoding = {}
frequency_encoding = {}
duration_encoding = {}
onset_encoding = {}
location_encoding={} 

In [7]:
all_symtom,nature,location,frequency,excitation,vas,onset,duration = extract_symptom_nlice(test)
data_dir = './data_preprossing/output/basic/min3/'
all_symtom_file = "./data_preprossing/output/basic/symptom_db_v1.json"
with open(all_symtom_file, "w") as fp:
    sorted_symptoms = sorted(all_symtom.keys())
    symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
    json.dump(all_symtom, fp, indent=4)
count = 1
for item in nature:
    nature_encoding[item] = count
    count += 1
nature_encoding_file = os.path.join(data_dir, "nature_encoding.json")
with open(nature_encoding_file, "w") as fp:
    json.dump(nature_encoding, fp, indent=4)

count = 1
for item in location:
    location_encoding[item] = count
    count += 1
location_encoding_file = os.path.join(data_dir, "body-parts-enc.json")
with open(location_encoding_file, "w") as fp:
    json.dump(location_encoding, fp, indent=4)

count = 1
for item in frequency:
    frequency_encoding[item] = count
    count += 1
frequency_encoding_file = os.path.join(data_dir, "frequency_encoding.json")
with open(frequency_encoding_file, "w") as fp:
    json.dump(frequency_encoding, fp, indent=4)

count = 1
for item in excitation:
    excitation_encoding[item] = count
    count += 1
excitation_encoding_file = os.path.join(data_dir, "excitation_encoding.json")
with open(excitation_encoding_file, "w") as fp:
    json.dump(excitation_encoding, fp, indent=4)

count = 1
for item in vas:
    vas_encoding[item] = count
    count += 1
vas_encoding_file = os.path.join(data_dir, "vas_encoding.json")
with open(vas_encoding_file, "w") as fp:
    json.dump(vas_encoding, fp, indent=4)

count = 1
for item in onset:
    onset_encoding[item] = count
    count += 1
onset_encoding_file = os.path.join(data_dir, "onset_encoding.json")
with open(onset_encoding_file, "w") as fp:
    json.dump(onset_encoding, fp, indent=4)

count = 1
for item in duration:
    duration_encoding[item] = count
    count += 1
duration_encoding_file = os.path.join(data_dir, "duration_encoding.json")
with open(duration_encoding_file, "w") as fp:
    json.dump(duration_encoding, fp, indent=4)

In [ ]:
output_path = "./data_preprossing/output/data/"
data1k = "./data_preprossing/output/data/total_1k.csv"
classlabel = [24, 29, 49, 38, 5, 3, 43, 48, 41, 37, 12, 31, 9, 42, 35, 28, 8, 14, 36, 4, 40, 19, 20, 21, 39, 53, 10, 25, 0, 44, 2, 17, 45]
symptom_columns = ['Index','GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
df = pd.read_csv(dataset_train,names=symptom_columns)
df.index.name = "Index"
df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=3 else np.NAN)
df['SYMPTOMS'].replace('', np.nan, inplace=True)
df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
df.dropna(subset=['SYMPTOMS'],inplace=True)
df = df.drop_duplicates()
label = "total_1k_min3"
output_file = os.path.join(output_path, "%s.csv" % label)
df.to_csv(output_file)